In [ ]:
Boyle.mk("env")
Boyle.activate("env")
Boyle.install({:emel, "~> 0.3.0"})
Boyle.install({:csv, "~> 2.0.0"})
alias Emel.Ml.DecisionTree, as: DT
alias Emel.Help.Model, as: Mdl

In [2]:
dataset = "../emel/notebooks/datasets/StudentsPerformance.csv"
    |> File.stream!()
    |> CSV.decode!(headers: true)
    |> Enum.to_list()
hd(dataset)

%{"gender" => "female", "lunch" => "standard", "math score" => "72", "parental level of education" => "bachelor's degree", "race/ethnicity" => "group B", "reading score" => "72", "test preparation course" => "none", "writing score" => "74"}

In [ ]:
categorizer =  Mdl.categorizer(["Fail", 40, "Sub-Pass", 45, "Pass"])

In [ ]:
parsed_dataset = Enum.map(
  dataset,
  fn %{"writing score" => score} = row ->
    grade = score
    |> Mdl.parse()
    |> categorizer.()
    row
    |> Map.drop(["math score", "reading score", "writing score"])
    |> Map.put("writing grade", grade)
  end
)

In [5]:
hd(parsed_dataset)

%{"gender" => "female", "lunch" => "standard", "parental level of education" => "bachelor's degree", "race/ethnicity" => "group B", "test preparation course" => "none", "writing grade" => "Pass"}

In [ ]:
{training_set, test_set} = Mdl.training_and_test_sets(parsed_dataset, 0.75)

In [ ]:
discrete_attributes = [
  "gender",
  "lunch",
  "parental level of education",
  "race/ethnicity",
  "test preparation course"
]
class = "writing grade"
f = DT.classifier(training_set, discrete_attributes, class)

In [8]:
observations = Enum.map(test_set, fn %{"writing grade" => grade} -> grade end)
Enum.take(observations, 5)

["Pass", "Pass", "Pass", "Pass", "Pass"]

In [9]:
predictions = Enum.map(test_set, fn row -> f.(row) end)
Enum.take(predictions, 5)

["Fail", "Pass", "Pass", "Pass", "Pass"]

In [10]:
Emel.Math.Statistics.similarity(predictions, observations)

0.9